In [1]:
import os
import sys
py_files_dir = os.path.abspath("../../../vampires_on_sky_calibration/programs/py_files/")
sys.path.insert(0, py_files_dir)

import em_gain
import numpy as np
import pandas as pd
import helper_functions as funcs

# Examing 04/28/2022 Data

In [2]:
# Define wavelengths
wavelengths = np.array(["625-50", "675-50", "725-50", "750-50", "775-50"])

# Read in CSV file
df = pd.read_csv("../../data/20220428/20220429_New_Masterdarks_Header_Parameters.csv")

for i, wavelength in enumerate(wavelengths):
    # Filter the DataFrame for the specific wavelength
    df_wavelength = df[(df["U_FILTER"] == wavelength)]
    
    # Further filter for each camera from the already filtered DataFrame
    df_cam1 = df_wavelength[df_wavelength["U_CAMERA"] == 1]
    df_cam2 = df_wavelength[df_wavelength["U_CAMERA"] == 2]

    # Extracting fluxes for each camera & FLC state
    FL1 = df_cam1["DARK_SUBTRACTED_TOTAL_MEDIAN_FLUX_FLC_1"].values
    FL2 = df_cam1["DARK_SUBTRACTED_TOTAL_MEDIAN_FLUX_FLC_2"].values
    FR1 = df_cam2["DARK_SUBTRACTED_TOTAL_MEDIAN_FLUX_FLC_1"].values
    FR2 = df_cam2["DARK_SUBTRACTED_TOTAL_MEDIAN_FLUX_FLC_2"].values

    # Adding together both FLC States
    FL = FL1 + FL2
    FR = FR1 + FR2

    # Assuming em_gain.calculate_em_gain_ratio returns normalized_fluxes and em_gain_ratios
    normalized_fluxes, em_gain_ratios = em_gain.calculate_em_gain_ratio(FL, FR)

    print("Number of Data Points: " + str(len(normalized_fluxes)))
    print(f"Median Normalized Flux for {wavelength}: {np.median(normalized_fluxes)}")
    print(f"Median EM Gain Ratio for {wavelength}: {np.median(em_gain_ratios)}")
    print()


Number of Data Points: 64
Median Normalized Flux for 625-50: -0.37148584094361287
Median EM Gain Ratio for 625-50: 2.182112369394222

Number of Data Points: 64
Median Normalized Flux for 675-50: -0.32687068726950047
Median EM Gain Ratio for 675-50: 1.972271901820175

Number of Data Points: 64
Median Normalized Flux for 725-50: -0.3032787251318735
Median EM Gain Ratio for 725-50: 1.870629599294518

Number of Data Points: 72
Median Normalized Flux for 750-50: -0.24548753479580876
Median EM Gain Ratio for 750-50: 1.650744237286565

Number of Data Points: 64
Median Normalized Flux for 775-50: -0.2948902912160366
Median EM Gain Ratio for 775-50: 1.8369183956949486



# Examine 01/28/2022 Standards

In [3]:
# Performing for just unpolarized standards first
all_normalized_fluxes = []
all_em_gain_ratios = []

print("Examining Unpolarized Standards")
unpol_csv_directory = "../../../vampires_on_sky_calibration/data/unpolarized/csv"
unpol_csv_files = funcs.load_all_files_from_directory(unpol_csv_directory, ".csv")

for csv_file in unpol_csv_files:
    normalized_fluxes, em_gain_ratios = em_gain.process_vampires_dpp_csv_file(csv_file)
    all_normalized_fluxes = np.concatenate((all_normalized_fluxes, normalized_fluxes))
    all_em_gain_ratios = np.concatenate((all_em_gain_ratios, em_gain_ratios))

print()
print("Median Normalized Fluxes (Unpolarized): " + str(np.median(all_normalized_fluxes)))
print("Median EM Gain Ratio (Unnolarized): " + str(np.median(all_em_gain_ratios)))
print()

# Resetting to look at polarized standards
all_normalized_fluxes = []
all_em_gain_ratios = []

print("Examining Polarized Standards")
pol_csv_directory = "../../../vampires_on_sky_calibration/data/polarized/csv"
pol_csv_files = funcs.load_all_files_from_directory(pol_csv_directory, ".csv")

for csv_file in pol_csv_files:
    normalized_fluxes, em_gain_ratios = em_gain.process_vampires_dpp_csv_file(csv_file)
    all_normalized_fluxes = np.concatenate((all_normalized_fluxes, normalized_fluxes))
    all_em_gain_ratios = np.concatenate((all_em_gain_ratios, em_gain_ratios))

# Convert lists to numpy arrays if they are not already
all_normalized_fluxes = np.array(all_normalized_fluxes)
all_em_gain_ratios = np.array(all_em_gain_ratios)

# Filter out NaN values
valid_indices = ~np.isnan(all_normalized_fluxes) & ~np.isnan(all_em_gain_ratios)
all_normalized_fluxes = all_normalized_fluxes[valid_indices]
all_em_gain_ratios = all_em_gain_ratios[valid_indices]

print()
print("Median Normalized Fluxes (Polarized): " + str(np.median(all_normalized_fluxes)))
print("Median EM Gain Ratio (Polarized): " + str(np.median(all_em_gain_ratios)))


Examining Unpolarized Standards
Number of Data Points: 20
Median Normalized Flux for HD173667: -0.044116104950356574
Median EM Gain Ratio for HD173667: 1.0923043177882397
Number of Data Points: 24
Median Normalized Flux for HD36819: -0.06962340595335258
Median EM Gain Ratio for HD36819: 1.149667207311924
Number of Data Points: 24
Median Normalized Flux for HD128750: -0.06068809047232204
Median EM Gain Ratio for HD128750: 1.1292181862008897
Number of Data Points: 211
Median Normalized Flux for ETACRV: -0.06427398306495762
Median EM Gain Ratio for ETACRV: 1.1373777834573546
Number of Data Points: 28
Median Normalized Flux for HD108767: -0.0477133023372153
Median EM Gain Ratio for HD108767: 1.1002078701867435
Number of Data Points: 40
Median Normalized Flux for HD173667: -0.03870248561647912
Median EM Gain Ratio for HD173667: 1.0805213711697501

Median Normalized Fluxes (Unpolarized): -0.06291537537449327
Median EM Gain Ratio (Unnolarized): 1.1342789620513443

Examining Polarized Standard